In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
import joblib

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import LeaveOneOut
import numpy as np

from statistics import mode


#read the csv file
df = pd.read_csv('data/stat_acc_V3_new.csv', sep=';')

### 1 - Répartition des données

In [2]:
# Sélection des 5 attributs qui nous intéresses pour l'apprentissage
df = df[['age', 'latitude', 'longitude','descr_athmo','descr_grav']]

# initialisation des variables X et Y contenant la data et la target de notre base de donnée
X = df.drop(['descr_grav'], axis=1)
Y = df['descr_grav']

# Séparation de la base de donnée en base de validation et d'entrainement
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Séparation de la base d'entrainement en base d'apprentissage et de validation
#X_app, X_test, y_app, y_test = train_test_split(X_train, y_train, test_size=0.16, random_state=42)

Nous nous limitons donc ici à 5 attributs pour la base de données : ['age', 'latitude', 'longitude','descr_athmo','descr_grav'].
Notre répartition est la suivante : base d'entraînement 80% et base de validation 20% 
Dans la suite nous allons donc diviser la base d'entraînement en base d'apprentissage et de test selon la répartition holdout et leave one out.

Dans la suite X et Y vont correspondre à X_train et y_train.

In [3]:
X = X_train
Y = y_train

In [4]:
X.head()

,age,latitude,longitude,descr_athmo
37295,51,43.8833,-0.500000,1
71390,56,42.6809,9.302510,1
51318,84,43.1000,5.883330,1
22545,50,43.6000,1.433330,1
36056,71,44.5000,0.966667,1


In [5]:
Y.head()

37295    1
71390    1
51318    3
22545    1
36056    3
Name: descr_grav, dtype: int64

#### 1.1 - Holdout Répartition

In [6]:
def holdout_repartition(X, Y, list, n):

    # Répétition 5 fois de holdout split
    for i in range(n):
        # Division des données en ensembles d'apprentissage et de tests
        X_app, X_test, y_app, y_test = train_test_split(X, Y, test_size=0.16, random_state=i)
        
        # Ajouter les ensembles d'apprentissage et de test à la liste
        list.append((X_app, X_test, y_app, y_test))
    

In [7]:
# Initialisation de la liste
holdout_app_test_sets = []

# Nombre de répétitions de holdout split
n = 5

# Ececution de la fonction holdout_repartition
holdout_repartition(X, Y, holdout_app_test_sets, n)

#### 1.2 - Leave One out Répartition

In [8]:
def leaveoneout_repartition(X, Y, list):
    
    # Création de l'objet leave one out
    loo = LeaveOneOut()
    
    for train_index, test_index in loo.split(leaveoneout_sample):     
        # Ajouter les ensembles d'apprentissage et de test et d'apprentissage à la liste
        list.append((X.iloc[train_index], X.iloc[test_index], Y.iloc[train_index], Y.iloc[test_index]))

In [9]:
# Récupération de 10% des données de chaque niveau de gravité
gravite_1 = df[df['descr_grav'] == 1].sample(frac=0.1, random_state=42)
gravite_2 = df[df['descr_grav'] == 2].sample(frac=0.1, random_state=42)
gravite_3 = df[df['descr_grav'] == 3].sample(frac=0.1, random_state=42)
gravite_4 = df[df['descr_grav'] == 4].sample(frac=0.1, random_state=42)

# Concaténation des échantillons sélectionnés
leaveoneout_sample = pd.concat([gravite_1, gravite_2, gravite_3, gravite_4])

#initialisation des variables X et Y du sample leaveoneout 
X_leaveoneout = df.drop(['descr_grav'], axis=1)
Y_leaveoneout = df['descr_grav']

# Initialisation de la liste
leaveoneout_app_test_sets = []

# Execution de leave one out
leaveoneout_repartition(X_leaveoneout, Y_leaveoneout, leaveoneout_app_test_sets)

In [10]:
# Liste pour stocker les valeurs d'accuracy
accuracy_list = [] 

# Boucle faisant la prédiction et l'évaluation pour chaque répartition holdout
for i, (X_app, X_test, y_app, y_test) in enumerate(leaveoneout_app_test_sets): 
   
   # Création du modèle KNN
   knn = KNeighborsClassifier(n_neighbors=5)

   # Apprentissage du modèle KNN
   knn.fit(X_app, y_app)

   # Prédiction des données de test
   y_pred = knn.predict(X_test)

   # Evaluation de la prédiction
   accuracy = accuracy_score(y_test, y_pred)
   
   # Ajout de l'accuracy à la liste
   accuracy_list.append(accuracy) 

In [11]:
# Calcul de la moyenne des accuracy
mean_accuracy = np.mean(accuracy_list)

print("Moyenne de l'accuracy :", mean_accuracy)

# Récupération de l'indice correspondant à l'accuracy la plus grande dans la liste holdout_app_test_sets
n_leaveoneout = np.argmax(accuracy_list)

print("Indice de le précision maximum dans la liste :", n_leaveoneout)

Moyenne de l'accuracy : 0.36569799022270505
Indice de le précision maximum dans la liste : 1


### 2 - Classification avec KNN

#### 2.1 - "From scratch"

In [ ]:
def distHamming(p1, p2):
    return np.sum(p1 != p2,axis=1)

def distManhattan(p1, p2):
    return np.sum(np.abs(p1 - p2),axis=1)

def predict_knn(X_train, y_train, X_test, k):
    neighbors = []

    for x in X_test.values:
        distances = distManhattan(x, X_train)
        #distances = distHamming(x, X_train)
        y_sorted = [y for _, y in sorted(zip(distances, y_train))]
        neighbors.append(list(y_sorted[:k]))
    return [mode(n) for n in neighbors]

df = pd.read_csv("data/stat_acc_V3_new.csv", sep=";")
#data = df.groupby("descr_grav").apply(lambda x: x.sample(frac=0.1))
selected_columns = ['age', 'latitude', 'longitude', 'descr_grav','descr_athmo','descr_cat_veh','descr_lum','descr_etat_surf']
df_selected = df[selected_columns].copy()

df_selected.dropna(inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'latitude', 'longitude', 'descr_athmo','descr_cat_veh','descr_lum','descr_etat_surf']],
                                                    df['descr_grav'], test_size=0.2, random_state=42)
k = 3
y_pred = predict_knn(X_train, y_train, X_test, k)

# TEST EVALUATION 
print(y_pred)
accuracy_score(y_test, y_pred)


# from sklearn.metrics import silhouette_score
# silhouette_avg = silhouette_score(y_test, y_pred)
# print("Silhouette Coefficient :", silhouette_avg)

# from sklearn.metrics import calinski_harabasz_score
# ch_score = calinski_harabasz_score(y_test, y_pred)
# print("Calinski-Harabasz Index :", ch_score)

# from sklearn.metrics import davies_bouldin_score
# db_score = davies_bouldin_score(y_test, y_pred)
# print("Davies-Bouldin Index :", db_score)

#### 2.2 - Bibliothèque scikit-learn

In [12]:
# Liste pour stocker les valeurs d'accuracy
accuracy_list = [] 

# Boucle faisant la prédiction et l'évaluation pour chaque répartition holdout
for i in range(5):   
   # Accès aux bases provenants de la répartition holdout
   X_app, X_test, y_app, y_test = holdout_app_test_sets[i]
   
   # Création du modèle KNN
   knn = KNeighborsClassifier(n_neighbors=5)

   # Apprentissage du modèle KNN
   knn.fit(X_app, y_app)

   # Prédiction des données de test
   y_pred = knn.predict(X_test)

   # Evaluation de la prédiction
   accuracy = accuracy_score(y_test, y_pred)
   
   # Ajout de l'accuracy à la liste
   accuracy_list.append(accuracy) 

In [13]:
# Calcul de la moyenne des accuracy
mean_accuracy = np.mean(accuracy_list)

print("Moyenne de l'accuracy :", mean_accuracy)

# Récupération de l'indice correspondant à l'accuracy la plus grande dans la liste holdout_app_test_sets
n = np.argmax(accuracy_list)

print(n)

Moyenne de l'accuracy : 0.4227668151920221
1


### 3 - Classification avec trois algorithmes de « haut niveau »

#### 3.1 - Classification avec SVM

In [31]:
def SVM_class(X,Y,old_model=None):
    #création d'un classifieur SVM
    svm = SVC()
    svm_params = {'kernel':('sigmoid', 'rbf'), 'C':[1, 10]}
    
    #aram_grid = {
    #    'C': [0.1, 1, 10, 100, 500, 1000, 2000, 5000, 10000], 
    #    'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    #}
    
    #svm_params = {'C': [1, 2]}
    
    #Initiation des paramètre possible
    #svm_params={                                                                     
    #        'C': [0.1, 1, 10, 100, 1000],
    #        'epsilon': [0.0001, 0.0005,  0.001, 0.005,  0.01, 0.05, 1, 5, 10],
    #        'gamma': [0.0001, 0.0005,  0.001, 0.005,  0.01, 0.05, 1, 5, 10]
    #}
    
    svm_grid = GridSearchCV(svm, svm_params, cv=3, n_jobs=4)
    svm_grid.fit(X, Y)
    best_svm = svm_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_svm, 'models/svm_model.pkl')
    else:
        if old_model.score(X,Y) < best_svm.score(X,Y):
            joblib.dump(best_svm, 'models/svm_model.pkl')
    # retourne le meilleur modèle
    return best_svm

In [32]:
# Accès à la répartition provenants de la répartition holdout
X_app, X_test, y_app, y_test = holdout_app_test_sets[n]

# Récupération d'échantillons du dataset
X_app_sample = X_app.sample(frac=0.001, random_state=42)
X_test_sample = X_test.sample(frac=0.001, random_state=42)
y_app_sample = y_app.sample(frac=0.001, random_state=42)
y_test_sample = y_test.sample(frac=0.001, random_state=42)

In [33]:
# Initialisation du modèle 
best_svm = None

# Selection du meilleur modèle SVM
best_svm = SVM_class(X_app, y_app, best_svm)

#### 3.2 - Classification avec Random Forest

In [26]:
def RF_class(X,Y,old_model=None):
    #création d'un classifieur Random Forest
    rf = RandomForestClassifier()
    rf_params = {'n_estimators':[10, 100, 1000], 'max_depth':[None, 10, 100]}
    #rf_params = {'n_estimators': [10, 100], 'max_depth': [None, 10]}
    rf_grid = GridSearchCV(rf, rf_params, cv=5)
    #rf_grid = GridSearchCV(rf, rf_params, cv=3)
    rf_grid.fit(X, Y)
    best_rf = rf_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_rf, 'models/rf_model.pkl')
    else:
        if old_model.score(X,Y) < best_rf.score(X,Y):
            joblib.dump(best_rf, 'models/rf_model.pkl')


    return best_rf

In [27]:
# Initialisation du modèle 
best_rf = None

# Selection du meilleur modèle SVM
best_rf = RF_class(X_app, y_app, best_rf)

#### 3.3 - Classification avec MLP

In [35]:
def MLP_class(X,Y,old_model=None):
    #création d'un classifieur MLP
    mlp = MLPClassifier()
    #mlp_params = {'hidden_layer_sizes':[(10,), (100,), (1000,)], 'activation':['relu', 'tanh', 'logistic']}
    mlp_params = {'hidden_layer_sizes': [(10,), (100,)], 'activation': ['relu', 'tanh']}
    mlp_grid = GridSearchCV(mlp, mlp_params, cv=5)
    mlp_grid.fit(X, Y)
    best_mlp = mlp_grid.best_estimator_

    #enregistrement du modèle avec les meilleurs paramètres
    #comparaison avec l'ancien modèle pour ne pas le remplacer si moins performant
    if old_model == None:
        joblib.dump(best_mlp, 'models/mlp_model.pkl')
    else:
        if old_model.score(X,Y) < best_mlp.score(X,Y):
            joblib.dump(best_mlp, 'models/mlp_model.pkl')

    return best_mlp

In [36]:
# Initialisation du modèle 
best_mlp = None

# Selection du meilleur modèle SVM
best_mlp = MLP_class(X_app, y_app, best_mlp)

#### 3.4 - Fusion classifieurs

In [37]:
#fusion des classifieurs

#vote majoritaire
voting_clf_maj = VotingClassifier(estimators=[('svm', best_svm), ('rf', best_rf), ('mlp', best_mlp)], voting='hard')
voting_clf_maj.fit(X, Y)

#vote pondéré
voting_clf_pond = VotingClassifier(estimators=[('svm', best_svm), ('rf', best_rf), ('mlp', best_mlp)], voting='soft')
voting_clf_pond.fit(X, Y)

#prediction sur un nouveau jeu de données (à modifier avec les jeux de données de test)
X_pred = X_test

#prediction vote majoritaire
Y_pred_maj = voting_clf_maj.predict(X_test)

#prediction vote pondéré
Y_pred_pond = voting_clf_pond.predict(X_test)

### 4 - Évaluation quantitative des résultats « supervisé »

#### 4.1 - Taux d’apprentissage

#### 4.2 - Matrice de confusion
 

#### 4.3 - Précision/Rappel

#### 4.4 - Courbe ROC